In [ ]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import os
root_folder = os.getcwd()

In [ ]:
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))
### Parameters for running computation
global Kb
global Kv
global Ka
global gamma
global KbT
global delT
global Kal
Kal=1
gamma=1
Ka=2
Kv=1
Kb=0.01 
Volume_t= 0.35*2* 3.14 * 4 / 3
Area_t=4*3.14
KbT=0.01
delT=0.01
sigma=np.sqrt(2*gamma*(KbT/delT))

In [ ]:
###Energy Calculations Area+Bending+Volume
def Energy_area(v,f):
    Area_new=cal_Area(v,f)
    Area_t=4*3.14
    Eenergy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Eenergy_Area
def Energy_volume(v,f):
    volume_old=cal_volume(v,f)
    volume_t=0.7*(4/3)*3.14
    Eenergy_volume=Kv*((volume_t-volume_old)**2)/volume_t
    return Eenergy_volume
def Energy_bending(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    #print(max(h_mean))
    #print(min(h_mean))
    c=len(v)
    #bending_modulus
    Eb=[] # bending_energy
    for i in range (c):
        Eb.append((Kb/2)*((h_mean[i])**2)*(area_voronoi[i]))
        Eb_array = np.array(Eb)
        total_EB=np.sum(Eb_array)
    return total_EB

In [ ]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

def cal_Area(v,f):
    dbl_area = igl.doublearea(v, f)    
    Area=np.sum(dbl_area)/2
    return Area
def volGrad(v,f):
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]
            FaceNorm=face_normal[k]
            vol_ij=vol_ij+((1/3)*Area*FaceNorm)
        volumegrad.append(vol_ij)
    volumegrad=np.array(volumegrad)
    return volumegrad

def areaGrad(v,f):
    n=igl.per_vertex_normals(v,f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    l = igl.cotmatrix(v, f) ###laplacian-operator
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    areagrad=2*n*h_mean[:,None]
    return areagrad
def cal_volume(v,f):
    Volume=0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volume=Volume+sum
    return Volume
def Local_Area(v,f):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    area_voronoi=m.diagonal()
    return area_voronoi    

adjacent_faces=adjacent_face(v,f)
k=adjacent_faces[0][2]
n = igl.per_vertex_normals(v, f)
face_normal=igl.per_face_normals(v,f,n)
face_normal[k]

def fun_volgrad(v,f):
    n = igl.per_vertex_normals(v, f)
    #vector=np.array([1,1,1])
    #norm=vector/np.linalg.norm(vector)

    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    volume_grad=face_normal*dbl_area[:,None]
    
    return volume_grad/3

In [ ]:
###Force from bending
def fun_ForceDensity(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    Lap_H=minv.dot(l.dot(h_mean/2))
    first_product=h_mean*(((h_mean/2)**2)-k)
    second_product=np.add(Lap_H,first_product)
    n = igl.per_vertex_normals(v, f)
    #normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    #Force_density=[] # Force_Density
    #Force_Nodal=[]

    #for i in range (len(v)):
    Force_density=2*Kb*(n*second_product[:,None])
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [ ]:
###Force from Area Constraints
def Force_Area(Area_t,grad_Area,Area_new):
    #grad_Area=volgrad(v,f)
    #Area_old=cal_volume(v,f)
    #Area_new=cal_Area(pos_new,f)
    Force_Area=-2*(Ka)*((Area_new-Area_t)/Area_t)*(grad_Area)
    return Force_Area

In [ ]:
###Force from Local Area Constraints
def Local_Force_Area(LArea_Original,grad_Area,LArea_Current):
    Local_Force_Area=-2*(Kal)*(((LArea_Current-LArea_Original)/LArea_Original))[:,None]*(grad_Area)
    return Local_Force_Area

In [ ]:
###Force from Volume Area Constraints
def Force_Volume(Volume_t,grad_Volume,volume_new):
    #grad_Volume=volgrad(v,f)
    #Volume_old=cal_volume(v,f)
    #Volume_new=cal_volume(pos_new,f)

    Force_Volume=-2*(Kv)*((volume_new-Volume_t)/Volume_t)*grad_Volume
    return Force_Volume

In [ ]:
###Dissipative Force
def fun_FD(v,f,velocity):
    adjacent_vertices=igl.adjacency_list(f)
    FDij_nodes=[]
    for i in range(len(v)):
        FD_ij=0
        for j in range(len(adjacent_vertices[i])):
            k=adjacent_vertices[i][j]
            vij=velocity[i]-velocity[k]
            rij=v[i]-v[k]
            #rij_distance=math.sqrt((rij[0][0]**2)+(rij[0][1]**2)+(rij[0][2]**2))
            rij_norm = rij/np.linalg.norm(rij)
            FD_ij=FD_ij+(-((gamma*np.dot(vij, rij)*rij_norm)))
        FDij_nodes.append(FD_ij) 
    return FDij_nodes

In [ ]:
####Random Forces
def fun_FR(v,f): 
    edges, fe, ef = igl.edge_topology(v, f)
    Fr=np.zeros((len(v),3))
    for e in range(len(edges)): 
        rij=v[edges[e][0]]-v[edges[e][1]]
        rij_norm = rij/np.linalg.norm(rij)
        gaussian=np.random.normal(0, 1, 1)
        Force=gaussian*rij_norm*sigma
        Fr[edges[e][0]]+=Force
        Fr[edges[e][1]]-=Force  
    return Fr

In [ ]:
def Total_Force(FB,FA,FV,FD,FR,FAL):
    Total_force=(FB+FA+FV+FB+FR+FAL)
    return Total_force

In [ ]:
#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_Area(v_init,f)
volume_new=cal_volume(v_init,f)
LArea_Original=Local_Area(v_init,f)

grad_A=areaGrad(v,f)

grad_Volume=volgrad(v,f)
grad_Volume=np.array(grad_Volume)
grad_Volume.shape

In [ ]:
#%%time
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))### Can be cahnged for Prolate
###Velocity Verlet Main 
dt=0.01
hdt=0.5*dt
iterations =50
#Setup
v=v_init
LArea_Current=Local_Area(v,f)
vel=np.zeros((len(v_init),3))
FD=np.array(fun_FD(v_init,f,vel))
FB=np.array(fun_ForceDensity(v_init,f))
FA=Force_Area(Area_t,grad_Area,Area_new)
FAL=Local_Force_Area(LArea_Original,grad_Area,LArea_Current)
FV=Force_Volume(Volume_t,grad_Volume,volume_new)
FR=fun_FR(v,f)
TF=Total_Force(FB,FA,FV,FD,FR,FAL)
### Calculation of Energy
totalEnergy=[]
EnergyArea=[]
EnergyVolume=[]
EnergyBending=[]
KineticEnergy=[]
#df=open('iter=0.01_100,gamma=100','w')
#df.write('Total Energy of the System, dt=0.01\n')
#new_file = open('FD.txt','w')
for i in range(iterations):
    KE=np.zeros((len(v_init),1))
    #Initial integration
    for j in range(len(v)):
        vel[j]+=0.5*TF[j]*dt
        v[j]+=vel[j]*dt
        KE[j]=np.sqrt((vel[j][0]**2)+(vel[j][1]**2)+(vel[j][2]**2))
    if i%5==0:
        totalEnergy.append(Energy_area(v,f)+Energy_bending(v,f)+Energy_volume(v,f))
        EnergyArea.append(Energy_area(v,f))
        EnergyVolume.append(Energy_volume(v,f))
        EnergyBending.append(Energy_bending(v,f))
        KineticEnergy.append(np.sum(KE))
        #print(pos_new)
        #totalEnergy.append(Energy_area(v,f)+Energy_bending(v,f)+Energy_volume(v,f))
        #EnergyArea.append(Energy_area(v,f))
        #EnergyVolume.append(Energy_volume(v,f))
        #EnergyBending.append(Energy_bending(v,f))
        #df.write(str(totalEnergy))
        #df.write('\n')
        mp.jupyter()
        p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)
        name = 'test'+str(i)
        #p.save(name)    
    #Force calculation
    Area_current=cal_Area(v,f)
    grad_Area_new=areaGrad(v,f)
    volume_current=cal_volume(v,f)
    grad_Volume_new=volGrad(v,f)
    LArea_Current=Local_Area(v,f)

    ###Add forces_new here
    FR=fun_FR(v,f)
    FD=np.array(fun_FD(v,f,vel))
    FB=np.array(fun_ForceDensity(v,f)) ##bending_force
    FA=Force_Area(Area_t,grad_Area_new,Area_current) 
    FAL=Local_Force_Area(LArea_Original,grad_Area_new,LArea_Current)
    #print(FAL)
    FV=Force_Volume(Volume_t,grad_Volume_new,volume_current)
    TF=Total_Force(FB,FA,FV,FD,FR,FAL)

    #     if i>1300 and i%5==0:
    #         ax1 = plt.figure().add_subplot(projection='3d')
    #         ax1.quiver(v[:,0], v[:,1],v[:,2], FA[:,0], FA[:,1], FA[:,2], length=0.2, normalize=True,color='g')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FA-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax2 = plt.figure().add_subplot(projection='3d')
    #         ax2.quiver(v[:,0], v[:,1],v[:,2], FV[:,0], FV[:,1], FV[:,2], length=0.2, normalize=True,color='r')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FV-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax3= plt.figure().add_subplot(projection='3d')
    #         ax3.quiver(v[:,0], v[:,1],v[:,2], FB[:,0], FB[:,1], FB[:,2], length=0.2, normalize=True,color='b')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FB-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    #         ax4= plt.figure().add_subplot(projection='3d')
    #         ax4.quiver(v[:,0], v[:,1],v[:,2], FD[:,0], FD[:,1], FA[:,2], length=0.2, normalize=True,color='y')
    #         filename = '/home/BU/dredwan1/Velocity Verlet/Gamma=100_Oblate/FD-images/'
    #         plt.savefig(filename + 'TEST_' + str(i) + '.png',dpi=600)
    
    ###Final Step
    for j in range(len(v)):
        vel[j]+=0.5*TF[j]*dt
        #print(vel_new)
        #df.close()

In [ ]:
time=np.linspace(0,50,10)
plt.plot(time,totalEnergy,'o-', color='red', label='Total Energy')
plt.plot(time,EnergyBending,'o-', label='Bending Energy')
plt.plot(time,EnergyVolume,'v-', label='Volume Energy')
plt.plot(time,EnergyArea,'o-', label='Area Energy')
#plt.plot(time,KineticEnergy,'v-', label='Kinetic Energy',color='k')
#xmin, xmax, ymin, ymax = plt.axis([xmin, xmax, ymin, ymax])
plt.legend(loc='upper left')
plt.title('time vs Energy')
plt.show()